In [14]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
intents_file = open('intents.json').read()
intents = json.loads(intents_file)

In [15]:
words=[]
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.', '|']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        #add documents in the corpus
        documents.append((word, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(documents)

[(['Hi'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['What', 'is', 'the', 'time', 'of', '|', '?'], 'time-course'), (['Time', 'of', '|'], 'time-course'), (['When', '|', 'starts', '?'], 'time-course'), (['At', 'what', 'time', '|', 'starts', '?'], 'time-course'), (['Who', 'is', 'the', 'Instructor', 'of', 'the', '|', 'course', '?'], 'course-instructor'), (['Who', 'provides', 'the', '|', 'course', '?'], 'course-instructor'), (['Who', 'is', 'the', 'Instructor', 'of', 'the', '|', '?'], 'course-instructor'), (['Who', 'provides', 'the', '|', '?'], 'course-instructor'), (['When', '|', "'s", 'lesson', 'is', 'provided', '?'], 'instructor-time'), (['When', '|', "'s", 'lesson', 'starts', '?'], 'instructor-time'), (['Time', 'of', '|', "'s", 'teacher', 'lesson'], 'instructor-time'), (['Schedule', 'of', '|', 'instructor'], 'instructor-time'), (['Are', 'there', 'lessons', 'on', '|'], 'time-course-info'), (['When', '|', "'s", 'lesson', 'starts', '?'], 'time-course-info'), (['Time', '

In [16]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

19 documents
5 classes ['course-instructor', 'greeting', 'instructor-time', 'time-course', 'time-course-info']
23 unique lemmatized words ["'s", 'are', 'at', 'course', 'hello', 'hi', 'hola', 'instructor', 'is', 'lesson', 'of', 'on', 'provided', 'provides', 'schedule', 'start', 'teacher', 'the', 'there', 'time', 'what', 'when', 'who']


In [17]:
# create the training data
training = []
# create empty array for the output
output_empty = [0] * len(classes)
# training set, bag of words for every sentence
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    word_patterns = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # create the bag of words array with 1, if word is found in current pattern
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training)
# create training and testing lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data is created")

Training data is created


In [18]:
# deep neural networds model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
# Compiling model. SGD with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#Training and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model is created")

Epoch 1/200
4/4 [==============================] - 0s 1ms/step - loss: 1.7092 - accuracy: 0.0544
Epoch 2/200
4/4 [==============================] - 0s 924us/step - loss: 1.6601 - accuracy: 0.1498
Epoch 3/200
4/4 [==============================] - 0s 856us/step - loss: 1.7301 - accuracy: 0.1098
Epoch 4/200
4/4 [==============================] - 0s 894us/step - loss: 1.6333 - accuracy: 0.0554
Epoch 5/200
4/4 [==============================] - 0s 1ms/step - loss: 1.6500 - accuracy: 0.1154
Epoch 6/200
4/4 [==============================] - 0s 744us/step - loss: 1.5173 - accuracy: 0.2796
Epoch 7/200
4/4 [==============================] - 0s 723us/step - loss: 1.5378 - accuracy: 0.3751
Epoch 8/200
4/4 [==============================] - 0s 836us/step - loss: 1.5332 - accuracy: 0.2586
Epoch 9/200
4/4 [==============================] - 0s 1ms/step - loss: 1.5163 - accuracy: 0.3284
Epoch 10/200
4/4 [==============================] - 0s 695us/step - loss: 1.5792 - accuracy: 0.2042
Epoch 11/200
4/